In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
itPostsRows = sc.textFile("../Data/Italian_Stack_Exchange/italianPosts.csv")
itPostsSplit = itPostsRows.map(lambda x: x.split("~"))
from pyspark.sql import Row
from datetime import datetime
def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S.%f")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None
    
def stringToPost(row):
  r = row.encode('utf8').split("~")
  return Row(
    toIntSafe(r[0]),
    toTimeSafe(r[1]),
    toIntSafe(r[2]),
    r[3],
    toIntSafe(r[4]),
    toTimeSafe(r[5]),
    toIntSafe(r[6]),
    toIntSafe(r[7]),
    r[8],
    toIntSafe(r[9]),
    toLongSafe(r[10]),
    toLongSafe(r[11]),
    long(r[12]))
from pyspark.sql.types import *
postSchema = StructType([
  StructField("commentCount", IntegerType(), True),
  StructField("lastActivityDate", TimestampType(), True),
  StructField("ownerUserId", LongType(), True),
  StructField("body", StringType(), True),
  StructField("score", IntegerType(), True),
  StructField("creationDate", TimestampType(), True),
  StructField("viewCount", IntegerType(), True),
  StructField("title", StringType(), True),
  StructField("tags", StringType(), True),
  StructField("answerCount", IntegerType(), True),
  StructField("acceptedAnswerId", LongType(), True),
  StructField("postTypeId", LongType(), True),
  StructField("id", LongType(), False)
  ])
rowRDD = itPostsRows.map(lambda x: stringToPost(x))
itPostsDFStruct = sqlContext.createDataFrame(rowRDD, postSchema)

from pyspark.sql.functions import *

In [4]:
from pyspark.sql.window import Window

postsDf = itPostsDFStruct
winDf =postsDf.filter(postsDf.postTypeId == 1).select(postsDf.ownerUserId, postsDf.id, postsDf.creationDate, lag(postsDf.id, 1).over(Window.partitionBy(postsDf.ownerUserId).orderBy(postsDf.creationDate)).alias("prev")).orderBy(postsDf.ownerUserId, postsDf.id).show()

+-----------+----+--------------------+----+
|ownerUserId|  id|        creationDate|prev|
+-----------+----+--------------------+----+
|          4|1637|2014-01-24 06:51:...|null|
|          8|   1|2013-11-05 20:22:...|null|
|          8| 112|2013-11-08 13:14:...|   1|
|          8|1192|2013-11-11 21:01:...| 112|
|          8|1276|2013-11-15 16:09:...|1192|
|          8|1321|2013-11-20 16:42:...|1276|
|          8|1365|2013-11-23 09:09:...|1321|
|         12|  11|2013-11-05 21:30:...|null|
|         12|  17|2013-11-05 22:17:...|  11|
|         12|  18|2013-11-05 22:34:...|  17|
|         12|  19|2013-11-05 22:38:...|  18|
|         12|  63|2013-11-06 17:54:...|  19|
|         12|  65|2013-11-06 18:07:...|  63|
|         12|  69|2013-11-06 19:41:...|  65|
|         12|  70|2013-11-06 20:35:...|  69|
|         12|  89|2013-11-07 19:22:...|  70|
|         12|  94|2013-11-07 20:42:...|  89|
|         12| 107|2013-11-08 08:27:...|  94|
|         12| 122|2013-11-08 20:55:...| 107|
|         

In [5]:
from pyspark.sql.window import Window

postsDf = itPostsDFStruct
winDf =postsDf.filter(postsDf.postTypeId == 1).select(postsDf.ownerUserId, postsDf.id, postsDf.creationDate, lag(postsDf.id, 1).over(Window.partitionBy(postsDf.ownerUserId).orderBy(postsDf.creationDate)).alias("prev"), lead(postsDf.id, 1).over(Window.partitionBy(postsDf.ownerUserId).orderBy(postsDf.creationDate)).alias("next")).orderBy(postsDf.ownerUserId, postsDf.id).show()

+-----------+----+--------------------+----+----+
|ownerUserId|  id|        creationDate|prev|next|
+-----------+----+--------------------+----+----+
|          4|1637|2014-01-24 06:51:...|null|null|
|          8|   1|2013-11-05 20:22:...|null| 112|
|          8| 112|2013-11-08 13:14:...|   1|1192|
|          8|1192|2013-11-11 21:01:...| 112|1276|
|          8|1276|2013-11-15 16:09:...|1192|1321|
|          8|1321|2013-11-20 16:42:...|1276|1365|
|          8|1365|2013-11-23 09:09:...|1321|null|
|         12|  11|2013-11-05 21:30:...|null|  17|
|         12|  17|2013-11-05 22:17:...|  11|  18|
|         12|  18|2013-11-05 22:34:...|  17|  19|
|         12|  19|2013-11-05 22:38:...|  18|  63|
|         12|  63|2013-11-06 17:54:...|  19|  65|
|         12|  65|2013-11-06 18:07:...|  63|  69|
|         12|  69|2013-11-06 19:41:...|  65|  70|
|         12|  70|2013-11-06 20:35:...|  69|  89|
|         12|  89|2013-11-07 19:22:...|  70|  94|
|         12|  94|2013-11-07 20:42:...|  89| 107|
